In [1]:
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
#Punto 4

In [ ]:
#Punto 5

In [2]:
#Punto 6
def sor(A,b,x0,w,itmax=1000,tolerancia=1e-14):
    x=x0.copy()
    sum_k= x.copy()
    residuo = np.linalg.norm( np.dot(A,x) - b)
    it=0
    while residuo>= tolerancia and it < itmax:
    
        sum_k[:]=0
        
        for i in range (A.shape[0]):
            for j in range (A.shape[1]):
                if i != j:
                     sum_k[i] += A[i][j] * x[j]
            x[i] = (1 - w) * x[i] + (w / A[i][i]) * (b[i] - sum_k[i])
        
        residuo = np.linalg.norm(np.dot(A, x) - b)
        it += 1
        
        if residuo > 1000:
            print('No calculable con SOR')
            x[:] = 0.
            break
    
        
    return x,it
def omega_optimo(A,b,x0,itmax=1000,tolerancia=1e-14):
    w=None
    iteracion_minima=float('inf')
    for w0 in range(1,2):
        metodo= sor(A,b,x0,itmax,tolerancia)
        iteracion=len(metodo)
        
        if iteracion< iteracion_minima:
            iteracion_minima=iteracion
            w=w0   
    return w
A = np.array([[3,-1,-1],[-1.,3.,1.],[2,1,4]])
b = np.array([1.,3.,7.])
w=omega_optimo(A,b,np.array([0,0,0]))
x=sor(A,b,np.array([0,0,0]),w)
x

No calculable con SOR


(array([1, 1, 1]), 2)